In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [11]:
max_results = 2000000
#city_set = [‘New+York’,’Chicago’,’San+Francisco’, ‘Austin’, ‘Seattle’, ‘Los+Angeles’, ‘Philadelphia’, ‘Atlanta’, ‘Dallas’, ‘Pittsburgh’, ‘Portland’, ‘Phoenix’, ‘Denver’, ‘Houston’, ‘Miami’, ‘Washington+DC’, ‘Boulder’]
city_set = ["Canada"]
columns = ["country", "job_title", "company_name", "location", "summary", "salary"]
sample_df = pd.DataFrame(columns = columns)

In [12]:
#for city in city_set:
for start in range(0, max_results, 10):
    ### https://ca.indeed.com/jobs?q=Data+Scientist&l=Canada&radius=25
    ### https://ca.indeed.com/jobs?q=technical&l=Canada&sort=date&radius=25
    page = requests.get("http://ca.indeed.com/jobs?q=Data+Analyst&l=Canada&sort=date&radius=25" + "&start=" + str(start))
    time.sleep(1.5)  #ensuring at least 1.5 second between page grabs
    soup_temp = BeautifulSoup(page.text, "lxml", from_encoding="utf-8")
    for div in soup_temp.find_all(name="div", attrs={"class":"row"}): 
        #specifying row num for index of job posting in dataframe
        num = (len(sample_df) + 1) 
        #creating an empty list to hold the data for each posting
        job_post = [] 

        #append country name
        job_post.append("Canada") 

        #grabbing job title
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"])

        #grabbing company name
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                job_post.append(b.text.strip())
        else:
            sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
            for span in sec_try:
                job_post.append(span.text.strip())

        #grabbing location name
        div_two = div.find(name="div",attrs={"class":"sjcl"})
        spans = div_two.find_all("span", attrs={"class": "location"})
        if len(spans) > 0 :
            for span in spans:
                job_post.append(span.text)
        else:
            divs = div_two.find_all("div", attrs={"class": "location"})
            if len(divs) > 0:
                for d in divs:
                    job_post.append(d.text)
            else :
                job_post.append("")

        #grabbing summary text
        summary_span = div.find_all("div", attrs={"class": "summary"})
        if len(summary_span) > 0:
            for span in summary_span:
                if(span.ul):
                    if(span.ul.find_all("li")):
                        lis = span.ul.find_all("li")
                        desc = ""
                        for li in lis:
                            desc = desc+li.text.strip()+'\n'
                        job_post.append(desc)
                else:
                    job_post.append("")
        else:
            job_post.append("")

        #grabbing salary
        salary_span = div.find_all(name="span", attrs={"class":"salaryText"})
        if len(salary_span) > 0:
            for span in salary_span:
                job_post.append(span.text.strip())
        else :
            job_post.append("Not Found")

        #appending list of job post info to dataframe at index num
#         print(job_post)
        sample_df.loc[num] = job_post

#saving sample_df as a local csv file — define your own local path to save contents 
sample_df.to_csv("Canada_all_roles.csv", encoding="utf-8")

C:\Users\harir\Anaconda3\lib\site-packages\bs4\__init__.py:177: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
